<a href="https://colab.research.google.com/github/gohmrmts/RoK_OCR_TOTALPOWER/blob/main/OCR_TOTAL_POWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !apt install tesseract-ocr libtesseract-dev tesseract-ocr-jpn
 !pip install pyocr
 !pip install Pillow

In [ ]:
# -*- coding: utf-8 -*-
import sys
from PIL import Image, ImageOps
import os

# 正規表現
import re

# トリミング前の画像の格納先
ORIGINAL_FILE_DIR = "/content/original/"
# トリミング後の画像の格納先
TRIMMED_FILE_DIR = "/content/trimed/"


if __name__ == '__main__':
  # もしトリミング後の画像の格納先が存在しなければ作る
  if os.path.isdir(TRIMMED_FILE_DIR) == False:
    os.makedirs(TRIMMED_FILE_DIR)
  if os.path.isdir(ORIGINAL_FILE_DIR) == False:
    os.makedirs(ORIGINAL_FILE_DIR)

**画像をoriginalにアップロード**

In [ ]:
# -*- coding: utf-8 -*-
import sys
from PIL import Image, ImageOps
import os

# 正規表現
import re


# 画像パスと、左上座標、右下座標を指定して、トリミングされたimageオブジェクトを返す。
def trim(path, left, top, right, bottom):
  im = Image.open(path)
  im_trimmed = im.crop((left,top,right,bottom))
  im_trimmed = ImageOps.invert(im_trimmed.convert('RGB'))
  return im_trimmed


if __name__ == '__main__':
  # for IPad
  # トリミングする左上の座標
  left, top = 1650, 550
  # トリミングする右上の座標
  right, bottom = 1850, 1400

  # 画像ファイル名を取得
  files = os.listdir(ORIGINAL_FILE_DIR)
  # 特定の拡張子のファイルだけを採用。実際に加工するファイルの拡張子に合わせる
  files = [name for name in files if name.split(".")[-1] in ["png", "PNG", "jpg", "JPG"]]

  for val in files:
    # オリジナル画像へのパス
    path = ORIGINAL_FILE_DIR + val
    # トリミングされたimageオブジェクトを取得
    im_trimmed = trim(path, left, top, right, bottom)
    # グレースケール
    # im_trimmed = im_trimmed.convert("L")
    # モノクローム
    # im_trimmed = im_trimmed.convert("1")

    # 画像を読みやすいように加工。
    size=im_trimmed.size
    img2=Image.new('RGB',size)
 
    border=110
 
    for x in range(size[0]):
        for y in range(size[1]):
            r,g,b=im_trimmed.getpixel((x,y))
            if r > border or g > border or b > border:
                r = 255
                g = 255
                b = 255
            im_trimmed.putpixel((x,y),(r,g,b))
 

    # トリミング後のディレクトリに保存。ファイル名の頭に"cut_"をつけている
    im_trimmed.save(TRIMMED_FILE_DIR+"cut_"+val, quality=95) #qualityは95より大きい値は推奨されていないらしい

In [ ]:
# OCR関連
from PIL import ImageFile 
ImageFile.LOAD_TRUNCATED_IMAGES = True

from PIL import Image

import pyocr

# spreadsheet関連 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# pyocr関連
tools = pyocr.get_available_tools()
tools
tool = tools[0]
tool.get_name()

'Tesseract (sh)'

In [ ]:
# 現在日時
import datetime

dt_now = datetime.datetime.now()
print(dt_now)

In [ ]:
## スプレッドシートの作成
filename = "RoK_kingdom_power_" + str(dt_now.year) + "/" + str(dt_now.month) + "/" + str(dt_now.day) + "/" + str(dt_now.hour) + ":" + str(dt_now.minute)
ss = gc.create(filename)
print(ss.title)

# シートを特定する（シート名で特定）
st = ss.worksheet("シート1")

In [ ]:
# 画像ファイル名を取得
files = os.listdir(TRIMMED_FILE_DIR)
# 特定の拡張子のファイルだけを採用。実際に加工するファイルの拡張子に合わせる
files = [name for name in files if name.split(".")[-1] in ["png", "PNG", "jpg", "JPG"]]
# ソート
files.sort()

In [ ]:
# OCR
strlist = []
for filename in files:
  img = Image.open(TRIMMED_FILE_DIR + filename)
  txt = tool.image_to_string(
      img,
      lang = "jpn",
      builder = pyocr.builders.TextBuilder(tesseract_layout=6)
      # builder = pyocr.builders.DigitBuilder()
      # builder = pyocr.builders.LineBoxBuilder(tesseract_layout=6)
      )
  strlist += txt.split("\n")
  print(filename)

In [ ]:
print(len(strlist))
strlist

In [ ]:
# データ整形
arr = []
for i in range(len(strlist)):
  if not strlist[i]:
    pass
  else :
    arr.append(strlist[i])

strlist.clear()

pattern = '\d'
for i in range(len(arr)) :
  if(re.match(pattern, arr[i][0])):
    arr[i] = arr[i].replace(",", "")
    arr[i] = arr[i].replace(".", "")
    arr[i] = arr[i].replace(" ", "")
    strlist.append(arr[i])


In [ ]:
import copy
strlist_row = copy.copy(strlist)
strlist

In [ ]:
print("---7の誤検出の修正---")
for k in range(5):
  err = []
  for i in range(2, len(strlist) - 1):
    if(int(strlist[i - 1]) >= int(strlist[i]) and int(strlist[i]) >= int(strlist[i + 1])):
      pass
    else:
      err.append(i)

  print(err)

  for i in range(len(err)):
    if i % 2 == 0:
      print(strlist[err[i]])
      if(strlist[err[i]][0] == "1"):
        if(strlist[err[i] - 1][0] == "7" or strlist[err[i] + 1][0] == "7"):
          print(err[i], " : ", strlist[err[i]], "->", "7" + strlist[err[i]][1:])
          strlist[err[i]] = "7" + strlist[err[i]][1:]
          

print("\n---そのほかの修正---")
if err:
  for i in range(len(err)):
    if i % 2 == 0:
      avg = int((int(strlist[err[i] - 1]) + int(strlist[err[i] + 1])) / 2)
      print(err[i], " : ", strlist[err[i]], "->", avg)
      strlist[err[i]] = str(avg)

In [ ]:
# 300になることを確認
len(strlist)

In [ ]:
# スプレッドシート書き込みのためにListに格納
data = []
for i in range(len(strlist)):
  data.append([i + 1, strlist_row[i].strip(), strlist[i].strip()])

In [ ]:
# 総戦力の計算
total = 0
total_row = 0
for i in range(len(strlist)):
  total += int(strlist[i])
  total_row += int(strlist_row[i])

data = [["TOTAL", total_row, total]] + data

In [ ]:
for i in range(len(data)) :
  if data[i][1] != data[i][2]:
    data[i] += ["fixed"]

In [ ]:
data

In [ ]:
# 2次元配列を一括で書き込み

ss.values_append("シート1", {"valueInputOption": "USER_ENTERED"}, {"values": data})